In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
from pathlib import Path
from datetime import datetime

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"�� Matplotlib version: {plt.matplotlib.__version__}")

✅ Libraries imported successfully!
📊 Pandas version: 2.3.1
�� Matplotlib version: 3.10.3


In [3]:
def load_csv_data(file_path, sample_size=None):
    """Load CSV data with optional sampling"""
    try:
        if file_path.endswith('.gz'):
            df = pd.read_csv(file_path, compression='gzip')
        else:
            df = pd.read_csv(file_path)
        
        if sample_size and len(df) > sample_size:
            df = df.sample(n=sample_size, random_state=42)
        
        print(f"✅ Loaded {file_path}")
        print(f"   📏 Shape: {df.shape}")
        print(f"   �� Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        
        return df
    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return None

def load_geojson_data(file_path):
    """Load GeoJSON data"""
    try:
        with open(file_path, 'r') as f:
            geojson_data = json.load(f)
        
        print(f"✅ Loaded {file_path}")
        print(f"   📍 Features: {len(geojson_data.get('features', []))}")
        
        return geojson_data
    except Exception as e:
        print(f"❌ Error loading {file_path}: {e}")
        return None

def display_data_info(df, name):
    """Display comprehensive information about a dataset"""
    print(f"\n📊 {name} Dataset Overview:")
    print(f"   📏 Shape: {df.shape}")
    print(f"   📋 Columns: {list(df.columns)}")
    print(f"   �� Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   ❓ Missing values: {df.isnull().sum().sum()}")
    print(f"   �� Duplicates: {df.duplicated().sum()}")
    
    # Show data types
    print(f"\n📝 Data Types:")
    print(df.dtypes.value_counts())
    
    # Show first few rows
    print(f"\n👀 First 3 rows:")
    print(df.head(3))
    print("\n" + "="*60)

In [4]:
print("🚀 Loading Airbnb datasets...\n")

# Load listings data (prefer uncompressed)
if Path('listings.csv').exists():
    listings_df = load_csv_data('listings.csv')
else:
    listings_df = load_csv_data('listings.csv.gz')

# Load reviews data (prefer uncompressed)
if Path('reviews.csv').exists():
    reviews_df = load_csv_data('reviews.csv')
else:
    reviews_df = load_csv_data('reviews.csv.gz')

# Load calendar data
calendar_df = load_csv_data('calendar.csv.gz')

# Load neighbourhoods data
neighbourhoods_df = load_csv_data('neighbourhoods.csv')

# Load GeoJSON data
neighbourhoods_geojson = load_geojson_data('neighbourhoods.geojson')

print("\n✅ All data files loaded!")

# Store all dataframes in a dictionary for easy access
datasets = {
    'Listings': listings_df,
    'Reviews': reviews_df,
    'Calendar': calendar_df,
    'Neighbourhoods': neighbourhoods_df
}

🚀 Loading Airbnb datasets...

✅ Loaded listings.csv
   📏 Shape: (21722, 18)
   �� Memory: 8.92 MB
✅ Loaded reviews.csv
   📏 Shape: (405687, 2)
   �� Memory: 25.92 MB
✅ Loaded calendar.csv.gz
   📏 Shape: (7928517, 7)
   �� Memory: 1497.87 MB
✅ Loaded neighbourhoods.csv
   📏 Shape: (11, 2)
   �� Memory: 0.00 MB
✅ Loaded neighbourhoods.geojson
   📍 Features: 11

✅ All data files loaded!


In [5]:
print("�� COMPREHENSIVE DATA OVERVIEW\n")

for name, df in datasets.items():
    if df is not None:
        display_data_info(df, name)
    else:
        print(f"❌ {name} Dataset: Failed to load\n")

print("\n�� Summary:")
successful_loads = sum(1 for df in datasets.values() if df is not None)
total_datasets = len(datasets)
print(f"   ✅ Successfully loaded: {successful_loads}/{total_datasets} datasets")
print(f"   📍 GeoJSON: {'✅' if neighbourhoods_geojson is not None else '❌'}")

�� COMPREHENSIVE DATA OVERVIEW


📊 Listings Dataset Overview:
   📏 Shape: (21722, 18)
   📋 Columns: ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm', 'license']
   �� Memory: 8.92 MB
   ❓ Missing values: 58220
   �� Duplicates: 0

📝 Data Types:
int64      7
float64    6
object     5
Name: count, dtype: int64

👀 First 3 rows:
      id                                               name  host_id  \
0  31094  Beautiful, spacious, central, renovated Penthouse   129976   
1  32379  155 m2 artist flat on Vesterbro, with 2 bathrooms   140105   
2  32841                Cozy flat for Adults/Quiet for kids   142143   

        host_name  neighbourhood_group              neighbourhood   latitude  \
0            Ebbe                  NaN  Vesterbro-Kongens Enghave  55.6666

In [6]:
if listings_df is not None:
    print("🏠 LISTINGS DATA ANALYSIS\n")
    
    # Basic statistics for numeric columns
    numeric_cols = listings_df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print("📈 Numeric Columns Statistics:")
        print(listings_df[numeric_cols].describe())
    
    # Analyze categorical columns
    categorical_cols = listings_df.select_dtypes(include=['object']).columns
    print(f"\n📋 Categorical Columns ({len(categorical_cols)}):")
    for col in categorical_cols[:10]:  # Show first 10
        unique_count = listings_df[col].nunique()
        print(f"   {col}: {unique_count} unique values")
        if unique_count <= 15:  # Show values if not too many
            print(f"      Values: {list(listings_df[col].unique())}")
    
    # Check for price-related columns
    price_cols = [col for col in listings_df.columns if 'price' in col.lower()]
    if price_cols:
        print(f"\n💰 Price-related columns: {price_cols}")
        for col in price_cols:
            if col in listings_df.columns:
                print(f"   {col}: {listings_df[col].describe()}")
    
    # Check for location columns
    location_cols = [col for col in listings_df.columns if any(x in col.lower() for x in ['lat', 'lon', 'neighbourhood', 'city'])]
    if location_cols:
        print(f"\n📍 Location-related columns: {location_cols}")
else:
    print("❌ Listings data not available")

🏠 LISTINGS DATA ANALYSIS

📈 Numeric Columns Statistics:
                 id       host_id  neighbourhood_group      latitude  \
count  2.172200e+04  2.172200e+04                  0.0  21722.000000   
mean   5.991839e+17  1.543218e+08                  NaN     55.680526   
std    5.172072e+17  1.800592e+08                  NaN      0.019103   
min    3.109400e+04  5.130000e+02                  NaN     55.615660   
25%    3.483605e+07  1.834102e+07                  NaN     55.666275   
50%    7.038996e+17  6.907512e+07                  NaN     55.681850   
75%    1.089093e+18  2.330120e+08                  NaN     55.695940   
max    1.382678e+18  6.851944e+08                  NaN     55.732470   

          longitude          price  minimum_nights  number_of_reviews  \
count  21722.000000   12879.000000    21722.000000       21722.000000   
mean      12.558699    1291.329296        4.613341          18.676319   
std        0.031187    1280.272641       17.624078          44.574748   
min

In [7]:
if reviews_df is not None:
    print("⭐ REVIEWS DATA ANALYSIS\n")
    
    # Basic statistics
    print("�� Reviews Statistics:")
    print(reviews_df.describe())
    
    # Check for rating columns
    rating_cols = [col for col in reviews_df.columns if 'rating' in col.lower() or 'score' in col.lower()]
    if rating_cols:
        print(f"\n⭐ Rating columns: {rating_cols}")
        for col in rating_cols:
            if col in reviews_df.columns:
                print(f"   {col}: {reviews_df[col].describe()}")
    
    # Check for date columns
    date_cols = [col for col in reviews_df.columns if 'date' in col.lower()]
    if date_cols:
        print(f"\n📅 Date columns: {date_cols}")
        for col in date_cols:
            try:
                reviews_df[col] = pd.to_datetime(reviews_df[col])
                print(f"   ✅ Converted {col} to datetime")
                print(f"   �� Range: {reviews_df[col].min()} to {reviews_df[col].max()}")
            except:
                print(f"   ❌ Could not convert {col} to datetime")
    
    # Show sample reviews if comments column exists
    if 'comments' in reviews_df.columns:
        print(f"\n�� Sample Reviews:")
        sample_reviews = reviews_df['comments'].dropna().head(2)
        for i, review in enumerate(sample_reviews, 1):
            preview = review[:150] + "..." if len(review) > 150 else review
            print(f"   Review {i}: {preview}")
else:
    print("❌ Reviews data not available")

⭐ REVIEWS DATA ANALYSIS

�� Reviews Statistics:
         listing_id
count  4.056870e+05
mean   2.612276e+17
std    4.168974e+17
min    3.109400e+04
25%    1.209163e+07
50%    3.296784e+07
75%    6.453624e+17
max    1.373734e+18

📅 Date columns: ['date']
   ✅ Converted date to datetime
   �� Range: 2010-07-25 00:00:00 to 2025-03-26 00:00:00


In [8]:
if calendar_df is not None:
    print("📅 CALENDAR DATA ANALYSIS\n")
    
    # Basic statistics
    print("📊 Calendar Statistics:")
    print(calendar_df.describe())
    
    # Check for date columns
    date_cols = [col for col in calendar_df.columns if 'date' in col.lower()]
    if date_cols:
        print(f"\n📅 Date columns: {date_cols}")
        for col in date_cols:
            try:
                calendar_df[col] = pd.to_datetime(calendar_df[col])
                print(f"   ✅ Converted {col} to datetime")
                print(f"   📅 Range: {calendar_df[col].min()} to {calendar_df[col].max()}")
            except:
                print(f"   ❌ Could not convert {col} to datetime")
    
    # Check for price columns
    price_cols = [col for col in calendar_df.columns if 'price' in col.lower()]
    if price_cols:
        print(f"\n💰 Price columns: {price_cols}")
        for col in price_cols:
            if col in calendar_df.columns:
                print(f"   {col}: {calendar_df[col].describe()}")
    
    # Check for availability columns
    avail_cols = [col for col in calendar_df.columns if 'available' in col.lower()]
    if avail_cols:
        print(f"\n✅ Availability columns: {avail_cols}")
        for col in avail_cols:
            if col in calendar_df.columns:
                print(f"   {col}: {calendar_df[col].value_counts()}")
else:
    print("❌ Calendar data not available")

📅 CALENDAR DATA ANALYSIS

📊 Calendar Statistics:
         listing_id  adjusted_price  minimum_nights  maximum_nights
count  7.928517e+06             0.0    7.928116e+06    7.928116e+06
mean   5.991821e+17             NaN    4.658711e+00    3.914670e+02
std    5.171949e+17             NaN    1.699077e+01    4.343241e+02
min    3.109400e+04             NaN    1.000000e+00    1.000000e+00
25%    3.483549e+07             NaN    2.000000e+00    2.100000e+01
50%    7.037323e+17             NaN    3.000000e+00    3.650000e+02
75%    1.089095e+18             NaN    4.000000e+00    7.000000e+02
max    1.382678e+18             NaN    1.111000e+03    1.125000e+03

📅 Date columns: ['date']
   ✅ Converted date to datetime
   📅 Range: 2025-03-23 00:00:00 to 2026-03-27 00:00:00

💰 Price columns: ['price', 'adjusted_price']
   price: count       7928517
unique         1625
top       $1,000.00
freq         501516
Name: price, dtype: object
   adjusted_price: count    0.0
mean     NaN
std      NaN
min  

In [9]:
print("🔍 DATA QUALITY ASSESSMENT\n")

for name, df in datasets.items():
    if df is not None:
        print(f"📊 {name} Dataset Quality:")
        
        # Missing values analysis
        missing_values = df.isnull().sum()
        missing_percentage = (missing_values / len(df)) * 100
        
        print("   ❓ Missing values by column:")
        for col in df.columns:
            if missing_values[col] > 0:
                print(f"      {col}: {missing_values[col]} ({missing_percentage[col]:.2f}%)")
        
        # Duplicate analysis
        duplicates = df.duplicated().sum()
        print(f"   🔄 Duplicate rows: {duplicates}")
        
        # Data types summary
        print(f"   📝 Data types: {df.dtypes.value_counts().to_dict()}")
        print()
    else:
        print(f"❌ {name} Dataset: Not available for quality assessment\n")

🔍 DATA QUALITY ASSESSMENT

📊 Listings Dataset Quality:
   ❓ Missing values by column:
      host_name: 5 (0.02%)
      neighbourhood_group: 21722 (100.00%)
      price: 8843 (40.71%)
      last_review: 2964 (13.65%)
      reviews_per_month: 2964 (13.65%)
      license: 21722 (100.00%)
   🔄 Duplicate rows: 0
   📝 Data types: {dtype('int64'): 7, dtype('float64'): 6, dtype('O'): 5}

📊 Reviews Dataset Quality:
   ❓ Missing values by column:
   🔄 Duplicate rows: 5549
   📝 Data types: {dtype('int64'): 1, dtype('<M8[ns]'): 1}

📊 Calendar Dataset Quality:
   ❓ Missing values by column:
      adjusted_price: 7928517 (100.00%)
      minimum_nights: 401 (0.01%)
      maximum_nights: 401 (0.01%)
   🔄 Duplicate rows: 0
   📝 Data types: {dtype('float64'): 3, dtype('O'): 2, dtype('<M8[ns]'): 1, dtype('int64'): 1}

📊 Neighbourhoods Dataset Quality:
   ❓ Missing values by column:
      neighbourhood_group: 11 (100.00%)
   🔄 Duplicate rows: 0
   📝 Data types: {dtype('float64'): 1, dtype('O'): 1}



In [10]:
print("�� ANALYSIS SUMMARY\n")

print("📊 Successfully loaded datasets:")
for name, df in datasets.items():
    status = "✅" if df is not None else "❌"
    shape = df.shape if df is not None else "N/A"
    print(f"   {status} {name}: {shape}")

print(f"\n📍 GeoJSON: {'✅' if neighbourhoods_geojson is not None else '❌'}")

print("\n🚀 Recommended Next Steps:")
print("1. 🔧 Data Cleaning: Handle missing values and data type conversions")
print("2. �� Data Merging: Join datasets on common keys (listing_id, neighbourhood)")
print("3. 📊 EDA: Explore relationships between variables")
print("4. 📈 Advanced Visualizations: Create interactive plots and maps")
print("5. 🎯 Feature Engineering: Create new features for analysis")
print("6. 🤖 Machine Learning: Build predictive models")
print("7. �� Insights: Generate actionable business insights")

print("\n💡 Key Analysis Opportunities:")
if listings_df is not None:
    print("   • Price analysis and market trends")
    print("   • Location-based insights")
    print("   • Property type and amenity analysis")
if reviews_df is not None:
    print("   • Sentiment analysis of reviews")
    print("   • Rating patterns and trends")
if calendar_df is not None:
    print("   • Seasonal pricing patterns")
    print("   • Availability analysis")

print("\n✅ Ready for advanced analysis! 🚀")

�� ANALYSIS SUMMARY

📊 Successfully loaded datasets:
   ✅ Listings: (21722, 18)
   ✅ Reviews: (405687, 2)
   ✅ Calendar: (7928517, 7)
   ✅ Neighbourhoods: (11, 2)

📍 GeoJSON: ✅

🚀 Recommended Next Steps:
1. 🔧 Data Cleaning: Handle missing values and data type conversions
2. �� Data Merging: Join datasets on common keys (listing_id, neighbourhood)
3. 📊 EDA: Explore relationships between variables
4. 📈 Advanced Visualizations: Create interactive plots and maps
5. 🎯 Feature Engineering: Create new features for analysis
6. 🤖 Machine Learning: Build predictive models
7. �� Insights: Generate actionable business insights

💡 Key Analysis Opportunities:
   • Price analysis and market trends
   • Location-based insights
   • Property type and amenity analysis
   • Sentiment analysis of reviews
   • Rating patterns and trends
   • Seasonal pricing patterns
   • Availability analysis

✅ Ready for advanced analysis! 🚀


In [17]:
for name, df in datasets.items():
    print(name)
    print(df.head())
  

Listings
      id                                               name  host_id  \
0  31094  Beautiful, spacious, central, renovated Penthouse   129976   
1  32379  155 m2 artist flat on Vesterbro, with 2 bathrooms   140105   
2  32841                Cozy flat for Adults/Quiet for kids   142143   
3  38499                     0 min. from everything in Cph.   122489   
4  39055            Stylish apartment in central Copenhagen   167511   

        host_name  neighbourhood_group              neighbourhood   latitude  \
0            Ebbe                  NaN  Vesterbro-Kongens Enghave  55.666602   
1            Lise                  NaN  Vesterbro-Kongens Enghave  55.672638   
2  Anders & Maria                  NaN                    sterbro  55.711760   
3       Christina                  NaN                   Indre By  55.684288   
4           Rikke                  NaN                Amager Vest  55.665070   

   longitude        room_type   price  minimum_nights  number_of_reviews  \
0

In [18]:
def remove_completely_nan_columns(df, name):
    """Remove columns that are completely NaN and report the changes"""
    if df is None:
        return df
    
    # Find columns that are completely NaN
    completely_nan_cols = df.columns[df.isnull().all()].tolist()
    
    if completely_nan_cols:
        print(f"��️ Removing {len(completely_nan_cols)} completely NaN columns from {name}:")
        for col in completely_nan_cols:
            print(f"   ❌ {col}")
        
        # Remove the columns
        df_cleaned = df.drop(columns=completely_nan_cols)
        
        print(f"✅ {name} shape after cleaning: {df_cleaned.shape}")
        print(f"   📏 Removed {len(completely_nan_cols)} columns")
        print()
        
        return df_cleaned
    else:
        print(f"✅ {name}: No completely NaN columns found")
        print()
        return df

# Clean all datasets
print("🧹 CLEANING COMPLETELY NaN COLUMNS\n")

# Clean each dataset
for name, df in datasets.items():
    if df is not None:
        datasets[name] = remove_completely_nan_columns(df, name)

# Update the individual dataframe variables
if 'Listings' in datasets:
    listings_df = datasets['Listings']
if 'Reviews' in datasets:
    reviews_df = datasets['Reviews']
if 'Calendar' in datasets:
    calendar_df = datasets['Calendar']
if 'Neighbourhoods' in datasets:
    neighbourhoods_df = datasets['Neighbourhoods']

print("✅ All datasets cleaned!")

🧹 CLEANING COMPLETELY NaN COLUMNS

��️ Removing 2 completely NaN columns from Listings:
   ❌ neighbourhood_group
   ❌ license
✅ Listings shape after cleaning: (21722, 16)
   📏 Removed 2 columns

✅ Reviews: No completely NaN columns found

��️ Removing 1 completely NaN columns from Calendar:
   ❌ adjusted_price
✅ Calendar shape after cleaning: (7928517, 6)
   📏 Removed 1 columns

��️ Removing 1 completely NaN columns from Neighbourhoods:
   ❌ neighbourhood_group
✅ Neighbourhoods shape after cleaning: (11, 1)
   📏 Removed 1 columns

✅ All datasets cleaned!


In [19]:
print("�� VERIFICATION AFTER CLEANING\n")

for name, df in datasets.items():
    if df is not None:
        print(f"�� {name} Dataset:")
        print(f"   📏 Shape: {df.shape}")
        print(f"   📋 Columns: {len(df.columns)}")
        
        # Check for any remaining completely NaN columns
        completely_nan_cols = df.columns[df.isnull().all()].tolist()
        if completely_nan_cols:
            print(f"   ⚠️ Still has {len(completely_nan_cols)} completely NaN columns")
        else:
            print(f"   ✅ No completely NaN columns remaining")
        
        # Show memory usage
        memory_mb = df.memory_usage(deep=True).sum() / 1024**2
        print(f"   �� Memory: {memory_mb:.2f} MB")
        print()
    else:
        print(f"❌ {name} Dataset: Not available\n")

print("🎯 Ready for further analysis!")

�� VERIFICATION AFTER CLEANING

�� Listings Dataset:
   📏 Shape: (21722, 16)
   📋 Columns: 16
   ✅ No completely NaN columns remaining
   �� Memory: 8.84 MB

�� Reviews Dataset:
   📏 Shape: (405687, 2)
   📋 Columns: 2
   ✅ No completely NaN columns remaining
   �� Memory: 6.19 MB

�� Calendar Dataset:
   📏 Shape: (7928517, 6)
   📋 Columns: 6
   ✅ No completely NaN columns remaining
   �� Memory: 1051.76 MB

�� Neighbourhoods Dataset:
   📏 Shape: (11, 1)
   📋 Columns: 1
   ✅ No completely NaN columns remaining
   �� Memory: 0.00 MB

🎯 Ready for further analysis!


In [21]:
print("🔍 CHECKING FOR LISTING ID COLUMNS\n")

# Check which datasets have listing_id columns
listing_id_cols = {}
for name, df in datasets.items():
    if df is not None:
        # Look for listing_id columns (case insensitive)
        id_cols = [col for col in df.columns if 'listing_id' in col.lower() or col.lower() == 'id']
        if id_cols:
            listing_id_cols[name] = id_cols
            print(f"✅ {name}: Found ID columns - {id_cols}")
        else:
            print(f"❌ {name}: No listing ID columns found")
    else:
        print(f"❌ {name}: Dataset not available")

print(f"\n📊 Datasets with ID columns: {list(listing_id_cols.keys())}")

🔍 CHECKING FOR LISTING ID COLUMNS

✅ Listings: Found ID columns - ['id']
✅ Reviews: Found ID columns - ['listing_id']
✅ Calendar: Found ID columns - ['listing_id']
❌ Neighbourhoods: No listing ID columns found

📊 Datasets with ID columns: ['Listings', 'Reviews', 'Calendar']


In [22]:
print("🔗 ANALYZING LISTING ID RELATIONSHIPS\n")

# Check for common listing IDs between datasets
if 'Listings' in listing_id_cols and listings_df is not None:
    listings_ids = set(listings_df[listing_id_cols['Listings'][0]].dropna())
    print(f"📊 Listings dataset has {len(listings_ids)} unique listing IDs")
    
    # Check Reviews dataset
    if 'Reviews' in listing_id_cols and reviews_df is not None:
        reviews_ids = set(reviews_df[listing_id_cols['Reviews'][0]].dropna())
        common_reviews = listings_ids.intersection(reviews_ids)
        print(f"⭐ Reviews dataset has {len(reviews_ids)} unique listing IDs")
        print(f"   🔗 Common with Listings: {len(common_reviews)} ({len(common_reviews)/len(listings_ids)*100:.1f}%)")
    
    # Check Calendar dataset
    if 'Calendar' in listing_id_cols and calendar_df is not None:
        calendar_ids = set(calendar_df[listing_id_cols['Calendar'][0]].dropna())
        common_calendar = listings_ids.intersection(calendar_ids)
        print(f"📅 Calendar dataset has {len(calendar_ids)} unique listing IDs")
        print(f"   🔗 Common with Listings: {len(common_calendar)} ({len(common_calendar)/len(listings_ids)*100:.1f}%)")
    
    print()

🔗 ANALYZING LISTING ID RELATIONSHIPS

📊 Listings dataset has 21722 unique listing IDs
⭐ Reviews dataset has 18758 unique listing IDs
   🔗 Common with Listings: 18758 (86.4%)
📅 Calendar dataset has 21722 unique listing IDs
   🔗 Common with Listings: 21722 (100.0%)



In [24]:
print("🔗 JOINING DATASETS ON LISTING ID (FIXED)\n")

# Start with listings as the base dataset
if listings_df is not None and 'Listings' in listing_id_cols:
    base_df = listings_df.copy()
    listings_id_col = listing_id_cols['Listings'][0]
    print(f"�� Base dataset: Listings ({base_df.shape})")
    print(f"   �� Using ID column: {listings_id_col}")
    
    # Join with Reviews
    if 'Reviews' in listing_id_cols and reviews_df is not None:
        reviews_id_col = listing_id_cols['Reviews'][0]
        print(f"\n🔄 Joining with Reviews dataset...")
        
        # Get actual columns from reviews dataset
        reviews_actual_cols = reviews_df.columns.tolist()
        print(f"   �� Available columns: {reviews_actual_cols}")
        
        # Create reviews summary based on available columns
        reviews_summary = reviews_df.groupby(reviews_id_col).agg({
            reviews_id_col: 'count'  # Count of reviews per listing
        }).rename(columns={reviews_id_col: 'total_reviews'})
        
        # Add rating columns if they exist
        rating_cols = [col for col in reviews_df.columns if 'rating' in col.lower() or 'score' in col.lower()]
        for col in rating_cols:
            reviews_summary[f'avg_{col}'] = reviews_df.groupby(reviews_id_col)[col].mean()
        
        # Add date columns if they exist
        date_cols = [col for col in reviews_df.columns if 'date' in col.lower()]
        for col in date_cols:
            reviews_summary[f'latest_{col}'] = reviews_df.groupby(reviews_id_col)[col].max()
            reviews_summary[f'earliest_{col}'] = reviews_df.groupby(reviews_id_col)[col].min()
        
        # Add comment length if comments column exists
        comment_cols = [col for col in reviews_df.columns if 'comment' in col.lower()]
        for col in comment_cols:
            reviews_summary[f'avg_{col}_length'] = reviews_df.groupby(reviews_id_col)[col].apply(
                lambda x: x.str.len().mean() if x.dtype == 'object' else 0
            )
        
        # Join with base dataset
        base_df = base_df.merge(
            reviews_summary, 
            left_on=listings_id_col, 
            right_index=True, 
            how='left'
        )
        
        print(f"   ✅ Joined Reviews summary: {base_df.shape}")
        print(f"   📊 Reviews columns added: {list(reviews_summary.columns)}")
    
    # Join with Calendar
    if 'Calendar' in listing_id_cols and calendar_df is not None:
        calendar_id_col = listing_id_cols['Calendar'][0]
        print(f"\n🔄 Joining with Calendar dataset...")
        
        # Create calendar summary (aggregate calendar data per listing)
        calendar_summary = calendar_df.groupby(calendar_id_col).agg({
            'available': lambda x: (x == 't').sum() if 'available' in calendar_df.columns else 0,  # Available days
        }).rename(columns={'available': 'available_days'})
        
        # Add total calendar days
        calendar_summary['total_calendar_days'] = calendar_df.groupby(calendar_id_col).size()
        calendar_summary['availability_rate'] = calendar_summary['available_days'] / calendar_summary['total_calendar_days']
        
        # Add price analysis if price column exists
        if 'price' in calendar_df.columns:
            calendar_summary['avg_price'] = calendar_df.groupby(calendar_id_col)['price'].apply(
                lambda x: pd.to_numeric(x.str.replace('$', '').str.replace(',', ''), errors='coerce').mean()
            )
        
        # Join with base dataset
        base_df = base_df.merge(
            calendar_summary, 
            left_on=listings_id_col, 
            right_index=True, 
            how='left'
        )
        
        print(f"   ✅ Joined Calendar summary: {base_df.shape}")
        print(f"   📊 Calendar columns added: {list(calendar_summary.columns)}")
    
    # Join with Neighbourhoods (if it has neighbourhood info)
    if neighbourhoods_df is not None:
        print(f"\n🔄 Adding neighbourhood information...")
        
        # Check if listings has neighbourhood column
        neighbourhood_cols = [col for col in base_df.columns if 'neighbourhood' in col.lower()]
        if neighbourhood_cols:
            neighbourhood_col = neighbourhood_cols[0]
            base_df = base_df.merge(
                neighbourhoods_df,
                left_on=neighbourhood_col,
                right_on='neighbourhood',
                how='left'
            )
            print(f"   ✅ Joined Neighbourhoods: {base_df.shape}")
    
    print(f"\n🎯 Final joined dataset shape: {base_df.shape}")
    print(f"📋 Total columns: {len(base_df.columns)}")
    
    # Store the joined dataset
    joined_df = base_df
    
else:
    print("❌ Cannot join datasets - Listings dataset or ID column not available")
    joined_df = None

🔗 JOINING DATASETS ON LISTING ID (FIXED)

�� Base dataset: Listings ((21722, 16))
   �� Using ID column: id

🔄 Joining with Reviews dataset...
   �� Available columns: ['listing_id', 'date']
   ✅ Joined Reviews summary: (21722, 19)
   📊 Reviews columns added: ['total_reviews', 'latest_date', 'earliest_date']

🔄 Joining with Calendar dataset...
   ✅ Joined Calendar summary: (21722, 23)
   📊 Calendar columns added: ['available_days', 'total_calendar_days', 'availability_rate', 'avg_price']

🔄 Adding neighbourhood information...
   ✅ Joined Neighbourhoods: (21722, 23)

🎯 Final joined dataset shape: (21722, 23)
📋 Total columns: 23


In [25]:
if joined_df is not None:
    print("📊 JOINED DATASET OVERVIEW\n")
    
    print(f"📏 Shape: {joined_df.shape}")
    print(f"📋 Columns: {len(joined_df.columns)}")
    print(f"�� Memory: {joined_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Show new columns from joins
    original_listings_cols = set(listings_df.columns)
    new_cols = [col for col in joined_df.columns if col not in original_listings_cols]
    
    if new_cols:
        print(f"\n🆕 New columns from joins ({len(new_cols)}):")
        for col in new_cols:
            missing_pct = (joined_df[col].isnull().sum() / len(joined_df)) * 100
            print(f"   • {col:<30} (Missing: {missing_pct:5.1f}%)")
    
    # Show sample of joined data
    print(f"\n👀 Sample of joined data:")
    print(joined_df.head(3))
    
    # Show data types
    print(f"\n📝 Data types:")
    print(joined_df.dtypes.value_counts())
    
else:
    print("❌ No joined dataset available")

📊 JOINED DATASET OVERVIEW

📏 Shape: (21722, 23)
📋 Columns: 23
�� Memory: 10.00 MB

🆕 New columns from joins (7):
   • total_reviews                  (Missing:  13.6%)
   • latest_date                    (Missing:  13.6%)
   • earliest_date                  (Missing:  13.6%)
   • available_days                 (Missing:   0.0%)
   • total_calendar_days            (Missing:   0.0%)
   • availability_rate              (Missing:   0.0%)
   • avg_price                      (Missing:   0.0%)

👀 Sample of joined data:
      id                                               name  host_id  \
0  31094  Beautiful, spacious, central, renovated Penthouse   129976   
1  32379  155 m2 artist flat on Vesterbro, with 2 bathrooms   140105   
2  32841                Cozy flat for Adults/Quiet for kids   142143   

        host_name              neighbourhood   latitude  longitude  \
0            Ebbe  Vesterbro-Kongens Enghave  55.666602  12.555283   
1            Lise  Vesterbro-Kongens Enghave  55.67263